In [ ]:
%pip3 uninstall torch torchvision torchaudio

UsageError: Line magic function `%uninstall` not found.


In [ ]:
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu126

^C
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from ultralytics import YOLO
from tqdm import tqdm

# Test verisi yolu
test_images_dir = 'datasets/carla-object-detection-dataset/images/test'

# Sınıf adları (data.yaml'dan aynı sırayla alınmalı)
class_names = ['Vehicle', 'Bike', 'Motorbike', 'Traffic Light', 'Traffic Sign']
target_class_index = class_names.index('Traffic Sign')  # ROC sadece bu sınıf için

# Eğitilmiş modeller
model_infos = [
    ("YOLOv8n", "runs/detect/train_YOLOv8n/weights/best.pt"),
    ("YOLOv9t", "runs/detect/train_YOLOv9t/weights/best.pt"),
    ("YOLOv9s", "runs/detect/train_YOLOv9s/weights/best.pt"),
    ("YOLOv9m", "runs/detect/train_YOLOv9m/weights/best.pt"),
    ("YOLOv9c", "runs/detect/train_YOLOv9c/weights/best.pt"),
    ("YOLOv9e", "runs/detect/train_YOLOv9e/weights/best.pt"),
    ("YOLOv10n", "runs/detect/train_YOLOv10n/weights/best.pt"),
    ("YOLOv11n", "runs/detect/train_YOLOv11n/weights/best.pt"),
    ("YOLOv12n", "runs/detect/train_YOLOv12n/weights/best.pt"),
]

# ROC için tüm modellerin skorlarını sakla
plt.figure(figsize=(10, 8))

for model_name, model_path in model_infos:
    model = YOLO(model_path)

    # Prediction sonuçları
    results = model.predict(source=test_images_dir, conf=0.001, iou=0.5, save=False, stream=True)

    y_true = []
    y_scores = []

    for result in tqdm(results, desc=f"{model_name} evaluating"):
        # Her image prediction
        boxes = result.boxes
        if boxes is None:
            continue

        for b in boxes:
            cls = int(b.cls.cpu().numpy())
            conf = float(b.conf.cpu().numpy())

            # Sadece hedef sınıfa bak
            if cls == target_class_index:
                y_true.append(1)
                y_scores.append(conf)
            else:
                y_true.append(0)
                y_scores.append(conf)

    # ROC eğrisi
    fpr, tpr, _ = roc_curve(y_true, y_scores)
    roc_auc = auc(fpr, tpr)

    # Çizim
    plt.plot(fpr, tpr, label=f'{model_name} (AUC = {roc_auc:.3f})')

# Görselleştir
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate (Recall)')
plt.title('ROC Curve Comparison for "Traffic Sign" Detection')
plt.legend(loc='lower right')
plt.grid(True)
plt.tight_layout()
plt.savefig('roc_curve_comparison.png')
plt.show()
